# Chapitre 5 - La data visualisation avec Python (1ère partie)

## 5.4 Les graphiques interactifs avec d’autres packages et outils

### 5.4.2 Création d’une visualisation avec Bokeh

Il s’agit ici de construire une visualisation qui sera disponible au format html, codée
en JavaScript grâce à BokehJS. Bokeh va générer ce fichier à partir de votre code en
Python.

Si nous nous intéressons aux données AirBnB :

In [ ]:
import pandas as pd

In [ ]:
# on importe les données
listing=pd.read_csv("../data/listing_extrait.csv")
listing["price"]=pd.to_numeric(listing["price"].str.replace("$","")\
                               .str.replace(",",""))

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show, output_file
# on crée les données
listing_chers = listing[listing["price"]>1000][["price","name","room_type","bedrooms"]]
# on définit le titre
TITLE = "Les logements les plus chers de Paris"
# on définit les outils que l’on veut afficher
tools = "pan, wheel_zoom, box_zoom, reset, save".split(',')
# on définit les informations devant apparaître lorsqu’on survole un point
hover = HoverTool(tooltips=[("Price", "@price"),
                            ("Description:", "@name"),
                            ("Type de logement:", "@room_type")
                           ])
tools.append(hover)
# on crée le graphique et on définit les axes
p = figure(tools=tools, toolbar_location="above",  plot_width=1200, title=TITLE)
p.xaxis.axis_label = "Prix"
p.yaxis.axis_label = "Nombre de chambres"

# on définit les données (un dictionnaire est attendu mais un DataFrame fonctionne)
source = ColumnDataSource(listing_chers)

# on ajoute les points sous forme de cercles
p.circle("price", "bedrooms", size=5, source=source,
line_color="black", fill_alpha=0.8)

# on sauvegarde le fichier html
output_file("logements-bokeh.html", title="AirBnB à Paris")

# on ouvre un onglet du navigateur pour afficher le résultat
show(p)

On a donc extrait un DataFrame avec les logements à plus de 1000 euros et on
les a représentés dans un graphique interactif. On voit sur la figure 5.29 que le graphique obtenu permet d’afficher les informations sur les points lorsqu’on les survole.

Les graphiques Bokeh peuvent ensuite très facilement s’intégrer dans des pages
web plus évoluées notamment grâce à l’environnement web Flask de Python.

### 5.4.3 Création d’une application web avec Bokeh

La représentation précédente permet de visualiser des données, mais celles-ci sont figées et stockées dans le fichier html créé. Bien souvent, on voudra aller plus loin et créer une application interactive sur les données.

Pour cela, on va utiliser Bokeh et on va concevoir un fichier Python avec l’extension .py qui va inclure notre application. La partie application basée sur un serveur de Bokeh se base sur un environnement web nommé Tornado.

La documentation de Bokeh comprend de nombreuses informations :
https://bokeh.pydata.org/en/latest/

L’exemple donné ici est un exemple simple avec lancement de l’application en
local. Nous allons donc créer un fichier appli.py, et une fois ce fichier complété, nous pourrons l’utiliser pour lancer notre application avec Bokeh.
Le code est dans un seul fichier avec la forme suivante :

In [ ]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.models import ColumnDataSource, Div, HoverTool, Column
from bokeh.models.widgets import Select
from bokeh.io import curdoc

# Récupération et préparation des données
# (on extrait uniquement les logements avec plus de 20 commentaires)
listing=pd.read_csv("../data/listing_extrait.csv", low_memory=False)
listing_chers = listing[listing["number_of_reviews"]>20]\
[["host_is_superhost","number_of_reviews", "price","name","room_type",
  "bedrooms","review_scores_rating"]]

# Définition des widgets (un outil de sélection en fonction de la colonne
# superhost)
superhost = Select(title="Super-host", value="All", options=["Vrai","Faux"])

# Définition de la source de données. Elle est vide et utilise un dictionnaire
source = ColumnDataSource(data=dict(nb_com=[], note_com=[], type_chambre=[],
                                    name=[], price=[]))
# Définition des informations à afficher lorsqu’on passe sur un point
TOOLTIPS=HoverTool(tooltips=[
    ("Nom", "@name"),
    ("Prix", "@price"),
    ("Nombre de commentaires", "@nb_com"),
    ("Note moyenne", "@note_com"),
    ("Type logement", "@type_chambre")
])

# construction de la figure et ajout des points à partir des données
p = figure(plot_height=600, plot_width=700,
           title="", toolbar_location=None, tools=[TOOLTIPS])
p.circle(x="nb_com", y="note_com", source=source, size=2)

# définition d’une fonction de mise à jour des données
def update() :
    if superhost.value == "Vrai":
        super_h="t"
    else:
        super_h="f"

    listing2=listing_chers[listing_chers["host_is_superhost"]==super_h]
    p.xaxis.axis_label = "Nombre de commentaires"
    p.yaxis.axis_label = "Note moyenne"
    # mise à jour des données
    source.data = dict(nb_com=listing2["number_of_reviews"],
                       note_com=listing2["review_scores_rating"],
                       type_chambre = listing2["room_type"],
                       name=listing2["name"],
                       price=listing2["price"]
                      )
# gestion des contrôles pour la mise à jour
# (on en a un seul dans notre cas Select)
controls = [superhost]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

# construction du layout pour l’affichage
inputs = Column(*controls, sizing_mode="fixed")
l = layout([inputs, p], sizing_mode="fixed")

# premier chargement des données
update()

# utilisation de curdoc() pour générer la dataviz
curdoc().add_root(l)
curdoc().title = "AirBnB"

Nous utilisons dans l’invite de commandes générale, ou dans celle d’Anaconda, la
commande suivante :

L’application est donc lancée sur le serveur Bokeh en local et vous pouvez y accéder en utilisant le lien : 

http://localhost:5006/appli

Cette application web permet de croiser le nombre de commentaires et l’évaluation moyenne des logements. Une liste déroulante offre la possibilité d’afficher les logements ayant le label super-host ou non. Par ailleurs, lorsqu’on passe sur chaque point les caractéristiques du logement apparaissent.

Le package **Dash** offre une solution alternative récente qui vous permet de construire des applications web en utilisant l’environnement de Plotly. Bokeh et Dash sont aujourd’hui des solutions en pleine évolution.